In [ ]:
import torch
import torch.nn as nn
import torchvision
resnet = torchvision.models.resnet.resnet50(pretrained=True)


class ConvBlock(nn.Module):
    """
    Helper module that consists of a Conv -> BN -> ReLU
    """

    def __init__(self, in_channels, out_channels, padding=1, kernel_size=3, stride=1, with_nonlinearity=True):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, padding=padding, kernel_size=kernel_size, stride=stride)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()
        self.with_nonlinearity = with_nonlinearity

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        if self.with_nonlinearity:
            x = self.relu(x)
        return x


class Bridge(nn.Module):
    """
    This is the middle layer of the UNet which just consists of some
    """

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.bridge = nn.Sequential(
            ConvBlock(in_channels, out_channels),
            ConvBlock(out_channels, out_channels)
        )

    def forward(self, x):
        return self.bridge(x)


class UpBlockForUNetWithResNet50(nn.Module):
    """
    Up block that encapsulates one up-sampling step which consists of Upsample -> ConvBlock -> ConvBlock
    """

    def __init__(self, in_channels, out_channels, up_conv_in_channels=None, up_conv_out_channels=None,
                 upsampling_method="conv_transpose"):
        super().__init__()

        if up_conv_in_channels == None:
            up_conv_in_channels = in_channels
        if up_conv_out_channels == None:
            up_conv_out_channels = out_channels

        if upsampling_method == "conv_transpose":
            self.upsample = nn.ConvTranspose2d(up_conv_in_channels, up_conv_out_channels, kernel_size=2, stride=2)
        elif upsampling_method == "bilinear":
            self.upsample = nn.Sequential(
                nn.Upsample(mode='bilinear', scale_factor=2),
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1)
            )
        self.conv_block_1 = ConvBlock(in_channels, out_channels)
        self.conv_block_2 = ConvBlock(out_channels, out_channels)

    def forward(self, up_x, down_x):
        """
        :param up_x: this is the output from the previous up block
        :param down_x: this is the output from the down block
        :return: upsampled feature map
        """
        x = self.upsample(up_x)
        x = torch.cat([x, down_x], 1)
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        return x


class UNetWithResnet50Encoder(nn.Module):
    DEPTH = 6

    def __init__(self, n_classes=2):
        super().__init__()
        resnet = torchvision.models.resnet.resnet50(pretrained=True)
        down_blocks = []
        up_blocks = []
        self.input_block = nn.Sequential(*list(resnet.children()))[:3]
        self.input_pool = list(resnet.children())[3]
        for bottleneck in list(resnet.children()):
            if isinstance(bottleneck, nn.Sequential):
                down_blocks.append(bottleneck)
        self.down_blocks = nn.ModuleList(down_blocks)
        self.bridge = Bridge(2048, 2048)
        up_blocks.append(UpBlockForUNetWithResNet50(2048, 1024))
        up_blocks.append(UpBlockForUNetWithResNet50(1024, 512))
        up_blocks.append(UpBlockForUNetWithResNet50(512, 256))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=128 + 64, out_channels=128,
                                                    up_conv_in_channels=256, up_conv_out_channels=128))
        up_blocks.append(UpBlockForUNetWithResNet50(in_channels=64 + 3, out_channels=64,
                                                    up_conv_in_channels=128, up_conv_out_channels=64))

        self.up_blocks = nn.ModuleList(up_blocks)

        self.out = nn.Conv2d(64, n_classes, kernel_size=1, stride=1)

    def forward(self, x, with_output_feature_map=False):
        pre_pools = dict()
        pre_pools[f"layer_0"] = x
        x = self.input_block(x)
        pre_pools[f"layer_1"] = x
        x = self.input_pool(x)

        for i, block in enumerate(self.down_blocks, 2):
            x = block(x)
            if i == (UNetWithResnet50Encoder.DEPTH - 1):
                continue
            pre_pools[f"layer_{i}"] = x

        x = self.bridge(x)

        for i, block in enumerate(self.up_blocks, 1):
            key = f"layer_{UNetWithResnet50Encoder.DEPTH - 1 - i}"
            x = block(x, pre_pools[key])
        output_feature_map = x
        x = self.out(x)
        del pre_pools
        if with_output_feature_map:
            return x, output_feature_map
        else:
            return x


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


In [ ]:
LEARNING_RATE = 5e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Learning rate is ", LEARNING_RATE)
import torch.optim as optim
device=DEVICE
filename=''
BATCH_SIZE =4
NUM_EPOCHS = 500
NUM_WORKERS = 1
UnetDepth=5
model = UNet(n_classes=1, padding=True,batch_norm=True, up_mode='upsample',depth=UnetDepth).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

PIN_MEMORY = True
LOAD_MODEL = False

In [ ]:
UNetWithResnet50Encoder()

UNetWithResnet50Encoder(
  (input_block): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (input_pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (down_blocks): ModuleList(
    (0): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)


In [ ]:
!pip install --upgrade --force-reinstall --no-deps albumentations

     |████████████████████████████████| 102kB 8.1MB/s 
  Found existing installation: albumentations 0.1.12
    Uninstalling albumentations-0.1.12:
      Successfully uninstalled albumentations-0.1.12


In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
from tqdm import tqdm
import numpy as np
from os import path
from skimage.io import imread
from torch.utils import data
import os
from torch.utils.data import Dataset
from PIL import Image
import cv2

In [ ]:
import torch
def training_function(loader, model, optimizer, loss_fn):
  loop = tqdm(loader)
  trainingLossSum=0
  for batch_idx, (data, targets) in enumerate(loop):
      data = data.to(device=DEVICE)
      targets = targets.float().to(device=DEVICE)
      predictions = model(data)#.permute(0, 1, 3,2)
      train_loss = loss_fn(predictions, targets)
      trainingLossSum+=train_loss                  
      optimizer.zero_grad()
      train_loss.backward()
      optimizer.step()
      # update tqdm loop
      loop.set_postfix(loss=train_loss.item())
  return trainingLossSum/len(loader)

In [ ]:
def validating_function(loader,model,optimizer,loss_fn):
  loop = tqdm(loader)
  valLossSum=0
  for batch_idx, (data, targets) in enumerate(loop):
      data = data.to(device=DEVICE)
      targets = targets.float().to(device=DEVICE)
      # print('targets shape is:',targets.size())
      with torch.no_grad(): # <1> dont build graph on validation
              predictions = model(data)
              val_loss = loss_fn(predictions, targets)
              valLossSum+=  val_loss   
              assert val_loss.requires_grad == False # <2> checks that our output requires_grad
              # args are forced to False inside this block
               
      optimizer.zero_grad()
      optimizer.step()
       # update tqdm loop
      loop.set_postfix(loss=val_loss.item())
  return valLossSum/len(loader)

Preparing data

In [ ]:
from torchvision import transforms
to_tensor = transforms.ToTensor()

In [ ]:
import os
from torch.utils.data import Dataset
import numpy as np
from os import path
from PIL import Image
import cv2

from PIL import Image

import torch
# from skimage.io import imread
# from torch.utils import data

def replaceInputName(mask_path,diseaseType):
  result = mask_path.index('IDRiD')
  newString=''

  if diseaseType==1:  #for Hard exudates
    newString= '_EX'
  elif diseaseType==2: #for hemorrages
    newString= '_HE'
  elif diseaseType==3:  
    newString= '_SE'
  else:
    newString= '_MA'

  toReplace= mask_path[result:result+8]
  mask_path=mask_path.replace(toReplace, toReplace+newString)
  mask_path=mask_path.replace('images','groundtruths')
  return mask_path

class CarvanaDataset(Dataset):
   
    def __init__(self, diseaseType,image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)
        self.diseaseType= diseaseType

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index])
        mask_path = replaceInputName(img_path,diseaseType)


        image= cv2.imread(img_path)
        image = np.array(image, dtype=np.float32)
        
        mask = cv2.imread(mask_path)
        mask = mask[:,:,2]
        mask[mask == 255.0] = 1.0
        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]
        return image, mask

In [ ]:
LEARNING_RATE = 5e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Learning rate is ", LEARNING_RATE)
import torch.optim as optim
device=DEVICE
filename=''
BATCH_SIZE =4
NUM_EPOCHS = 500
NUM_WORKERS = 1
UnetDepth=5
IMAGE_HEIGHT = 512 
IMAGE_WIDTH = 512
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


 
PIN_MEMORY = True
LOAD_MODEL = False

In [ ]:
lossFunction=4

In [ ]:
LEARNING_RATE=0
if lossFunction==1:
    LEARNING_RATE = 5e-5
    ALPHA = 0.5
    BETA = 0.5
    # https://arxiv.org/abs/1706.05721
    #optimise segmentation on imbalanced medical datasets
    # in the case of α=β=0.5 the Tversky index simplifies to be the same as the Dice coefficient, which is also equal to the F1 score
    #modify beta increasing, so FN are punished harder
    class TverskyLoss(nn.Module):
        def __init__(self, weight=None, size_average=True):
            super(TverskyLoss, self).__init__()

        def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
            
            #comment out if your model contains a sigmoid or equivalent activation layer
            inputs = F.sigmoid(inputs)       
            
            #flatten label and prediction tensors
            inputs = inputs.view(-1)
            targets = targets.view(-1)
            
            #True Positives, False Positives & False Negatives
            TP = (inputs * targets).sum()    
            FP = ((1-targets) * inputs).sum()
            FN = (targets * (1-inputs)).sum()
          
            Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
            
            return 1 - Tversky
    loss_fn = TverskyLoss().to(DEVICE)
    filename="TverskyLossModel"+str(ALPHA)+str(BETA)+"resnetEbaneo"+"DEPTH"+str(UnetDepth)+".pth.tar"
    print("TverskyLoss selected")
    print(filename)
elif lossFunction==2:
      LEARNING_RATE = 5e-5
      #combo loss :  https://arxiv.org/abs/1805.02798
      #Combo loss is a combination of Dice Loss and a modified Cross-Entropy function that,
      # like Tversky loss, has additional constants which penalise either false positives or false negatives more respectively.
      ##PyTorch
      ALPHA = 0.5 # < 0.5 penalises FP more, > 0.5 penalises FN more
      CE_RATIO = 0.5 #weighted contribution of modified CE loss compared to Dice loss
      e=1e-7
      class ComboLoss(nn.Module):
        def __init__(self, weight=None, size_average=True):
            super(ComboLoss, self).__init__()

        def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA):
            
            #flatten label and prediction tensors
            inputs = inputs.view(-1)
            targets = targets.view(-1)
            
            #True Positives, False Positives & False Negatives
            intersection = (inputs * targets).sum()    
            dice = (2. * intersection + smooth) / (inputs.sum() + targets.sum() + smooth)
            
            inputs = torch.clamp(inputs, e, 1.0 - e)       
            out = - (ALPHA * ((targets * torch.log(inputs)) + ((1 - ALPHA) * (1.0 - targets) * torch.log(1.0 - inputs))))
            weighted_ce = out.mean(-1)
            combo = (CE_RATIO * weighted_ce) - ((1 - CE_RATIO) * dice)
            
            return combo
      loss_fn = ComboLoss().to(DEVICE)
      filename="ComboLossModel"+str(alpha)+str(1-alpha)+"resnetEbaneo"+"DEPTH"+str(UnetDepth)+".pth.tar"
      print("Combo loss selected")
      print(filename)
elif lossFunction == 3:
      LEARNING_RATE = 5e-5
      #Focal Loss
      #for combatting extremely imbalanced datasets https://arxiv.org/abs/1708.02002

      ALPHA = 0.8 #DONT MODIFY
      GAMMA = 2  #DONT MODIFY

      class FocalLoss(nn.Module):
          def __init__(self, weight=None, size_average=True):
              super(FocalLoss, self).__init__()

          def forward(self, inputs, targets, alpha=ALPHA, gamma=GAMMA, smooth=1):
              
              #comment out if your model contains a sigmoid or equivalent activation layer
              inputs = F.sigmoid(inputs)       
              
              #flatten label and prediction tensors
              inputs = inputs.view(-1)
              targets = targets.view(-1)
              
              #first compute binary cross-entropy 
              BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
              BCE_EXP = torch.exp(-BCE)
              focal_loss = alpha * (1-BCE_EXP)**gamma * BCE
                            
              return focal_loss
      loss_fn = FocalLoss().to(DEVICE)
      filename="FocalLossModelA"+str(ALPHA)+"resnetEbaneo"+"DEPTH"+str(UnetDepth)+".pth.tar"
      print("Focal loss selected")
      print(filename)
elif lossFunction==4:
    #focal tversky loss
    #includes the gamma correction from focal loss
    ALPHA = 0.2 #penalizes FP 
    BETA = 0.8 #penalizes FN
    GAMMA = 2
    LEARNING_RATE = 5e-5
    alpha=ALPHA
    betha=1-alpha
    class FocalTverskyLoss(nn.Module):
        def __init__(self, weight=None, size_average=True):
            super(FocalTverskyLoss, self).__init__()

        def forward(self, inputs, targets, smooth=1, alpha=ALPHA, beta=BETA, gamma=GAMMA):
            
            #comment out if your model contains a sigmoid or equivalent activation layer
            inputs = F.sigmoid(inputs)       
            
            #flatten label and prediction tensors
            inputs = inputs.view(-1)
            targets = targets.view(-1)
            
            #True Positives, False Positives & False Negatives
            TP = (inputs * targets).sum()    
            FP = ((1-targets) * inputs).sum()
            FN = (targets * (1-inputs)).sum()
            
            Tversky = (TP + smooth) / (TP + alpha*FP + beta*FN + smooth)  
            FocalTversky = (1 - Tversky)**gamma
                          
            return FocalTversky
    loss_fn = FocalTverskyLoss().to(DEVICE)
    filename="FocalTverskyLossModel"+str(alpha)+str(betha)+"gamma"+str(GAMMA)+"resnetEbaneo"+str(UnetDepth)+".pth.tar"
    print("tversky loss selected"+str(alpha)+str(betha)+str(GAMMA))
    print(filename)

else:
    loss_fn = nn.BCEWithLogitsLoss()
    filename="BCEWithLogitsLossLossModelSuperAugmentedresnetEbaneo"+"DEPTH"+str(UnetDepth)+".pth.tar"
    print('bcewithlogits')
    print(filename)

In [ ]:
#Hard Exudates
#diseaseType=1

#Hemorrhages
#diseaseType=2

#Soft Exudates
diseaseType=3

#Microa.
#diseaseType=4

root = '/content/drive/MyDrive/Colab Notebooks/'
folder=''
diseaseFolder=''


if diseaseType==1:  #for Hard exudates
  %cd /content/drive/My Drive/Colab Notebooks/hard exudates new unet
  diseaseFolder='hard exudates new unet'


elif diseaseType==2: #for hemorrages
  diseaseFolder='hemorrages full new unet'
  %cd /content/drive/My Drive/Colab Notebooks/hemorrages full new unet



elif diseaseType==3: #for soft exudates
  diseaseFolder='soft exudates new unet'
  %cd /content/drive/My Drive/Colab Notebooks/soft exudates new unet
  filename='softExudates'+filename
else: #microaneurysms
  diseaseFolder='microaneurysms new unet'
  %cd /content/drive/My Drive/Colab Notebooks/microaneurysms new unet


folder='/content/drive/My Drive/Colab Notebooks/'+diseaseFolder
TRAIN_IMG_DIR = root+diseaseFolder+"/images/training"
TRAIN_MASK_DIR = root+diseaseFolder+"/groundtruths/training"
VAL_IMG_DIR = root+diseaseFolder+"/images/test"
VAL_MASK_DIR = root+diseaseFolder+"/groundtruths/test"


filename = "LR"+str(LEARNING_RATE)+(diseaseFolder+filename).replace(" ", "")
print(filename)



/content/drive/My Drive/Colab Notebooks/soft exudates new unet
LR5e-05softexudatesnewunetsoftExudatesLR5e-05softexudatesnewunetsoftExudatesLR5e-05hardexudatesnewunetLR5e-05softexudatesnewunetsoftExudatesLR5e-05hemorragesfullnewunetFocalTverskyLossModel0.20.8gamma2resnetEbaneo5.pth.tar


In [ ]:
from torch.utils.data import DataLoader

def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CarvanaDataset(
                diseaseType,
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform

    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CarvanaDataset(
                diseaseType,
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform

    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader

training

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2


In [ ]:
train_transform = A.Compose(
    [
        #A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Rotate(limit=35, p=1.0),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.1),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=5, border_mode=0, p=0.5),
        A.GridDistortion(p=0.3),
        #A.ChannelShuffle(),
        A.OneOf([
                          A.MotionBlur(p=0.1),
                          A.OpticalDistortion(p=0.1)
                          ,A.GaussNoise(p=0.1)
                          ], p=0.1),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)
val_transforms = A.Compose(
    [
        #A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
        A.Normalize(
            mean=[0.0, 0.0, 0.0],
            std=[1.0, 1.0, 1.0],
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ],
)

In [ ]:
import torch.optim as optim




train_loader, val_loader = get_loaders(
    TRAIN_IMG_DIR,
    TRAIN_MASK_DIR,
    VAL_IMG_DIR,
    VAL_MASK_DIR,
    BATCH_SIZE,
    train_transform,
    val_transforms,
    NUM_WORKERS,
    PIN_MEMORY,
)

In [ ]:
import torch
def save_checkpoint(state, filename):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [ ]:
optim = torch.optim.Adam(model.parameters())
import torch.nn as nn
bestLoss=1000
#loss function
#loss_fn= nn.BCEWithLogitsLoss()
trainingLoss=[]
validationLoss=[]

n=0


for epoch in range(NUM_EPOCHS):
  n+=1
  print('epoch: ',n)
  tloss=training_function(train_loader,model,optim,loss_fn)
  print('tloss='+str(tloss))
  trainingLoss.append(tloss)
  vloss=validating_function(val_loader,model,optim,loss_fn)
  validationLoss.append(vloss)
  print('vloss: '+ str(vloss))
  if bestLoss> vloss:
    bestLoss=vloss
    checkpoint = {
      "state_dict": model.state_dict(),
      "optimizer":optimizer.state_dict(),
    }
    save_checkpoint(checkpoint,filename)

In [ ]:
def save_predictions_as_imgs(
    loader, model, folder="saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds=model(x)
            print(torch.unique(preds))
            preds = torch.sigmoid(preds)
            preds[preds>0.5] = 1.0
            preds[preds<=0.5] = 0.0
            pre = 255.0 * np.array(preds.permute(0, 2, 3, 1).to("cpu").detach())
            print(pre.shape)
            print("Now starting")
            for i in range(BATCH_SIZE):
              print(pre[i].shape)
              abc= "/content/drive/My Drive/softexudates/pred_"+str(idx)+" value "+str(i)+".jpg"
              writeStatus = cv2.imwrite(abc, pre[i])
              if writeStatus is True:
                  print("image written")
              else:
                  print("problem") # or raise exception, handle problem, etc.

In [ ]:
load_checkpoint(torch.load("/content/drive/MyDrive/Colab Notebooks/soft exudates new unet/newUNet_SE_FocalTverskyLossModel0.20.82UnetEbaneo.pth.tar"), model)
save_predictions_as_imgs(
        val_loader, model, folder="/content/drive/MyDrive/Colab Notebooks/saved_images", device=DEVICE
    )

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, auc, precision_recall_curve
import matplotlib.pyplot as plt

In [ ]:
def evaluate_precission_recall(
    loader, model, device="cuda"
):
    
    model.eval()
    predictions_array=[]
    targets_array=[]
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            targets = y.cpu().data.numpy()
            targets = torch.from_numpy(targets)
            targets = targets.float().unsqueeze(1).to(device=DEVICE)
            #print('targets shape: ',targets.shape)
            targets = targets.cpu().data.numpy()
            targets= targets.astype(np.float16)
            preds=model(x)
            #print('preds no pad',preds.shape)
            #preds= F.pad(input=preds, pad=(0, 1, 0, 1), mode='constant', value=0) # 4 layers
            #preds= F.pad(input=preds, pad=(0, 10, 8, 2), mode='constant', value=0) # resnet 50
            preds= F.pad(input=preds, pad=(0, 9, 8, 1), mode='constant', value=0) # 5 layers
            #print('preds padd ',preds.shape)
            preds = torch.sigmoid(preds)
            preds =preds.cpu().data.numpy()
            preds = preds.astype(np.float16)

            
            predictions_array=np.concatenate((predictions_array, preds.flatten()), axis=None)
            targets_array=np.concatenate((targets_array, targets.flatten()), axis=None)


    precision, recall, th = precision_recall_curve(targets_array,predictions_array)
    return precision, recall, th

In [ ]:
def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])

In [ ]:
load_checkpoint(torch.load("/content/drive/MyDrive/Colab Notebooks/microaneurysms new unet/LR1e-06microaneurysmsnewunetFocalTverskyLossModel0.20.8gamma2resnetEbaneo5.pth.tar"), model)
a,b,c = evaluate_precission_recall(val_loader,model)

In [ ]:
precision =a
recall =b

In [ ]:
a

In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support, auc, precision_recall_curve
import matplotlib.pyplot as plt

plt.figure()
plt.plot(recall,precision)
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('AUPRC Microaneurysm')
plt.savefig('ma_focal_AUPRC_08_02_2', dpi=300, bbox_inches='tight')
plt.show()
print(f'AUC Precision Recall {str}: {auc(recall, precision)}')

In [ ]:
from torchsummary import summary
model = UNetWithResnet50Encoder(n_classes=1).cuda()
summary(model, input_size=(3, 4,569, 569))